<a href="https://colab.research.google.com/github/angelinawong1210/VietnameseFakeNews/blob/main/FakeNews_SVM_Poly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
truenews = pd.read_csv('/content/Fake.csv')
fakenews = pd.read_csv('/content/Real.csv')

In [ ]:
truenews['True/Fake']='True'
fakenews['True/Fake']='Fake'

In [ ]:
# Combine the 2 DataFrames into a single data frame
news = pd.concat([truenews, fakenews])
news["Article"] = news["tittle"] + news["text"]
news.sample(frac = 1) #Shuffle 100%

,tittle,text,domain,topic,True/Fake,Article
57,Lao động nước ngoài ở Việt Nam phải đóng bảo h...,Chính phủ vừa ban hành Nghị định quy định chi...,vnexpress.net,Kinh tế,Fake,Lao động nước ngoài ở Việt Nam phải đóng bảo h...
41,Những dấu hiệu nhận biết nhà bạn đang có ma và...,Dấu hiệu nhận biết nhà có ma. Các khung giờ 1...,tinvn.info,Đời sống,True,Những dấu hiệu nhận biết nhà bạn đang có ma và...
71,"Lấy chồng thực vật, 35 tuổi vẫn còn zin, 1 đêm...","Trang đã từng yêu vài người, nhưng lần nào ch...",tinvn.info,Đời sống,True,"Lấy chồng thực vật, 35 tuổi vẫn còn zin, 1 đêm..."
51,Xã hội đen Đài Loan mở tiệm mỳ miễn phí cho ng...,\n Xã hội đen Italy dụ người nghèo hủy hoại ch...,vnexpress.net,Đời sống,Fake,Xã hội đen Đài Loan mở tiệm mỳ miễn phí cho ng...
62,Chấp nhận đổ vỏ cho con nhỏ bạn thân và cái kế...,Tối đang ngồi nhậu với mấy thằng bạn thì con ...,tinvn.info,Đời sống,True,Chấp nhận đổ vỏ cho con nhỏ bạn thân và cái kế...
...,...,...,...,...,...,...
34,FastGo quyết khô máu với Grab!,Trải nghiệm những giải đấu thể thao quốc tế tr...,tinvn.info,Kinh tế,True,FastGo quyết khô máu với Grab!Trải nghiệm nhữn...
76,"Cứ tưởng đào được trứng khủng long, người đàn ...",\n Không ai có thể ngờ được rằng bên trong nhữ...,tinvn.info,Đời sống,True,"Cứ tưởng đào được trứng khủng long, người đàn ..."
53,vũ khí kết nối người trẻ của Tổng thống Indonesia,WEF ASEAN đón số lượng kỷ lục lãnh đạo quốc g...,vnexpress.net,Chính trị,Fake,vũ khí kết nối người trẻ của Tổng thống Indone...
146,"Muốn mua Iphone 14 tặng bạn gái, thanh niên đậ...",Công an huyện Việt Yên đang tiến hành lập hồ s...,NaN,An ninh,True,"Muốn mua Iphone 14 tặng bạn gái, thanh niên đậ..."


In [ ]:
import nltk 
nltk.download('stopwords')
#Data Cleaning
from nltk.corpus import stopwords
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def process_text(s):

    # Check string to see if they are a punctuation
    nopunc = [char for char in s if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Convert string to lowercase and remove stopwords
    clean_string = [word for word in nopunc.split()]
    return clean_string

In [ ]:
# Tokenize the Article
#rerun, takes LOOOONG
news['Clean Text'] = news['Article'].apply(process_text)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
bow_transformer = CountVectorizer(analyzer=process_text).fit(news['Clean Text'])

print(len(bow_transformer.vocabulary_)) #Total vocab words


298


In [ ]:
#Bag-of-Words (bow) transform the entire DataFrame of text
news_bow = bow_transformer.transform(news['Clean Text'])

In [ ]:
print('Shape of Sparse Matrix: ', news_bow.shape)
print('Amount of Non-Zero occurences: ', news_bow.nnz)

Shape of Sparse Matrix:  (299, 298)
Amount of Non-Zero occurences:  299


In [ ]:
sparsity = (100.0 * news_bow.nnz / (news_bow.shape[0] * news_bow.shape[1]))
print('sparsity: {}'.format(round(sparsity)))

sparsity: 0


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(news_bow)
news_tfidf = tfidf_transformer.transform(news_bow)
print(news_tfidf.shape)

(299, 298)


In [ ]:
#Train SVM model
from sklearn.svm import SVC
fakenews_detect_model = SVC(kernel = 'poly', random_state = 0).fit(news_tfidf, news['True/Fake'])


In [ ]:
#Model Evaluation
predictions = fakenews_detect_model.predict(news_tfidf)
print(predictions)

['True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True' 'True'
 'True

In [ ]:
from sklearn.metrics import classification_report
print (classification_report(news['True/Fake'], predictions))


              precision    recall  f1-score   support

        Fake       1.00      1.00      1.00       150
        True       1.00      1.00      1.00       149

    accuracy                           1.00       299
   macro avg       1.00      1.00      1.00       299
weighted avg       1.00      1.00      1.00       299



In [ ]:
from sklearn.model_selection import train_test_split

news_train, news_test, text_train, text_test = train_test_split(news['Article'], news['True/Fake'], test_size=0.3)

print(len(news_train), len(news_test), len(news_train) + len(news_test))

209 90 299


In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=process_text)),  
    ('tfidf', TfidfTransformer()),  
    ('classifier', SVC(kernel = 'poly', random_state = 0)),  
])
pipeline.fit(news_train,text_train)

Pipeline(steps=[('bow',
                 CountVectorizer(analyzer=<function process_text at 0x7f09c16bcd30>)),
                ('tfidf', TfidfTransformer()),
                ('classifier', SVC(kernel='poly', random_state=0))])

In [ ]:
prediction = pipeline.predict(news_test)

In [ ]:
print(classification_report(prediction,text_test))

              precision    recall  f1-score   support

        Fake       0.97      0.53      0.68        74
        True       0.30      0.94      0.45        16

    accuracy                           0.60        90
   macro avg       0.64      0.73      0.57        90
weighted avg       0.85      0.60      0.64        90

